# Layers and Blocks

## Construct a MLP

In [1]:
from mxnet import nd
from mxnet.gluon import nn

net = nn.Sequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))

### Forward

In [2]:
x = nd.random.uniform(shape=(2, 20))
net.initialize()
net(x)


[[ 0.09543003  0.04614332 -0.00286653 -0.07790346 -0.05130243  0.02942039
   0.08696645 -0.0190793  -0.04122177  0.05088576]
 [ 0.0769287   0.03099705  0.00856576 -0.04467198 -0.0692684   0.09132432
   0.06786594 -0.06187843 -0.03436674  0.04234695]]
<NDArray 2x10 @cpu(0)>

## Implement the Same MLP with A Custom Block

In [3]:
class MLP(nn.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Dense(256, activation='relu')  
        self.output = nn.Dense(10) 

    def forward(self, x):
        return self.output(self.hidden(x))

### Forward 

In [4]:
net = MLP()
net.initialize()
net(x)


[[ 0.0036223   0.00633331  0.03201144 -0.01369375  0.10336448 -0.03508019
  -0.00032164 -0.01676024  0.06978628  0.01303308]
 [ 0.03871716  0.02608212  0.03544959 -0.02521311  0.11005434 -0.01430662
  -0.03052465 -0.03852826  0.06321152  0.0038594 ]]
<NDArray 2x10 @cpu(0)>

## Implement My Sequential Block

In [5]:
class MySequential(nn.Block):
    def __init__(self, **kwargs):
        super(MySequential, self).__init__(**kwargs)

    def add(self, block):
        # _children is an OrderedDict to store all sub-blocks. 
        # When calling the initialize function, the system automatically 
        # initializes all members of _children.
        self._children[block.name] = block

    def forward(self, x):
        # OrderedDict guarantees that members will be 
        # traversed in the order they were added.
        for block in self._children.values():
            x = block(x)
        return x

### Forward 

In [6]:
net = MySequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))
net.initialize()
net(x)


[[ 0.07787763  0.00216402  0.016822    0.0305988  -0.00702019  0.01668715
   0.04822846  0.0039432  -0.09300035 -0.04494302]
 [ 0.08891079 -0.00625484 -0.01619132  0.03807179 -0.01451489  0.02006173
   0.0303478   0.02463485 -0.07605447 -0.04389168]]
<NDArray 2x10 @cpu(0)>

## Blocks with Code

In [7]:
class FancyMLP(nn.Block):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        # Random weight parameters are not iterated during training
        self.rand_weight = self.params.get_constant(
            'rand_weight', nd.random.uniform(shape=(20, 20)))
        self.dense = nn.Dense(20, activation='relu')

    def forward(self, x):
        x = self.dense(x)
        # This layer will not be updated during training.
        x = nd.relu(nd.dot(x, self.rand_weight.data()) + 1)
        # Reuse the fully connected layer. 
        x = self.dense(x)
        while x.norm().asscalar() > 1:
            x /= 2
        if x.norm().asscalar() < 0.8:
            x *= 10
        return x.sum()

### Forward

In [8]:
net = FancyMLP()
net.initialize()
net(x)


[25.522684]
<NDArray 1 @cpu(0)>

## Mix Things Together

In [9]:
class NestMLP(nn.Block):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential()
        self.net.add(nn.Dense(64, activation='relu'),
                     nn.Dense(32, activation='relu'))
        self.dense = nn.Dense(16, activation='relu')
    def forward(self, x):
        return self.dense(self.net(x))
    
chimera = nn.Sequential()
chimera.add(NestMLP(), nn.Dense(20), FancyMLP())    
chimera.initialize()
chimera(x)


[30.518446]
<NDArray 1 @cpu(0)>